In [29]:
%run data_cleaning.py
df_cleaned.head()

,Age,T Stage,N Stage,6th Stage,Grade,A Stage,Tumor Size (mm),Estrogen Status,Progesterone Status,Regional Node Examined,Regional Node Positive,Survival Months,Status
0,68,0,0,0,2,1,4,1,1,24,1,60,0
1,50,1,1,2,1,1,35,1,1,14,5,62,0
2,58,2,2,4,1,1,63,1,1,14,7,75,0
3,58,0,0,0,2,1,18,1,1,2,1,84,0
4,47,1,0,1,2,1,41,1,1,3,1,50,0


In [30]:
numerical_columns = [col for col in df_cleaned.columns if col != "Tumor Size (mm)"]

from sklearn.preprocessing import RobustScaler

# Apllying RobustScaler to scale orginal numerical features
df_cleaned[numerical_columns] = RobustScaler().fit_transform(df_cleaned[numerical_columns])
df_cleaned.head()

,Age,T Stage,N Stage,6th Stage,Grade,A Stage,Tumor Size (mm),Estrogen Status,Progesterone Status,Regional Node Examined,Regional Node Positive,Survival Months,Status
0,1.000000,-1.0,0.0,-0.5,1.0,0.0,4,0.0,0.0,1.0,-0.333333,-0.516129,0.0
1,-0.285714,0.0,1.0,0.5,0.0,0.0,35,0.0,0.0,0.0,1.000000,-0.451613,0.0
2,0.285714,1.0,2.0,1.5,0.0,0.0,63,0.0,0.0,0.0,1.666667,-0.032258,0.0
3,0.285714,-1.0,0.0,-0.5,1.0,0.0,18,0.0,0.0,-1.2,-0.333333,0.258065,0.0
4,-0.500000,0.0,0.0,0.0,1.0,0.0,41,0.0,0.0,-1.1,-0.333333,-0.838710,0.0


In [31]:
#random forest regression model for tumour size
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np



data1 = remove_outliers_tumor_size(df_cleaned)
# data1=df.copy()
x = data1.drop(["Status", "Survival Months", "Tumor Size (mm)", "6th Stage", "T Stage", "Regional Node Examined"], axis=1)
y = data1["Tumor Size (mm)"]

x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.1)


param_grid = {
    'n_estimators': [100, 200, 300, 500, 1000],
    'max_depth': [None, 3, 5, 10, 20],
    # 'min_samples_split': [2, 5, 10],
    # 'min_samples_leaf': [1, 2, 4],
    # 'bootstrap': [True, False],
    # 'max_leaf_nodes': [None, 10, 20, 30],
    # 'min_impurity_decrease': [0.0, 0.1, 0.2],
    # 'oob_score': [True, False],
    # 'warm_start': [True, False],
    # 'ccp_alpha': [0.0, 0.1, 0.2],
    # 'max_samples': [None, 10, 20, 30],
    'criterion': ['squared_error', 'absolute_error', 'friedman_mse'],
}

best_params_list={'n_estimators':[], 'max_depth':[], 'min_samples_split':[], 'min_samples_leaf':[], 'bootstrap':[], 'max_leaf_nodes':[], 'min_impurity_decrease':[], 'oob_score':[], 'warm_start':[], 'ccp_alpha':[], 'max_samples':[], 'criterion':[]}
for j in range(5):
    rf = RandomForestRegressor(random_state=j)
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                        cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
    grid_search.fit(x_train, y_train)
    best_params = grid_search.best_params_
    print("Best parameters for iteration", j+1, ":", best_params)
    # Append the best parameters to the list
    for key in best_params.keys():
        best_params_list[key].append(best_params[key])
# Calculate the most popular best parameters

for key, values in best_params_list.items():
    try:
        mean = np.mean(values)
        # print(key, mean)
        best_params_list[key] = mean
    except TypeError:
        mode = max(set(values), key=values.count)
        best_params_list[key] = mode

print("Best parameters:", best_params_list)






Fitting 5 folds for each of 75 candidates, totalling 375 fits


Best parameters for iteration 1 : {'criterion': 'squared_error', 'max_depth': 3, 'n_estimators': 100}
Fitting 5 folds for each of 75 candidates, totalling 375 fits
Best parameters for iteration 2 : {'criterion': 'squared_error', 'max_depth': 3, 'n_estimators': 100}
Fitting 5 folds for each of 75 candidates, totalling 375 fits
Best parameters for iteration 3 : {'criterion': 'squared_error', 'max_depth': 3, 'n_estimators': 300}
Fitting 5 folds for each of 75 candidates, totalling 375 fits
Best parameters for iteration 4 : {'criterion': 'squared_error', 'max_depth': 3, 'n_estimators': 500}
Fitting 5 folds for each of 75 candidates, totalling 375 fits
Best parameters for iteration 5 : {'criterion': 'squared_error', 'max_depth': 3, 'n_estimators': 1000}
Best parameters: {'n_estimators': 400.0, 'max_depth': 3.0, 'min_samples_split': nan, 'min_samples_leaf': nan, 'bootstrap': nan, 'max_leaf_nodes': nan, 'min_impurity_decrease': nan, 'oob_score': nan, 'warm_start': nan, 'ccp_alpha': nan, 'max_

C:\Users\jeanl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\jeanl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [33]:
rf = RandomForestRegressor(random_state=42, bootstrap=True, max_depth = 3, n_estimators=400, criterion='squared_error')

rf.fit(x_train, y_train)

y_val_pred = rf.predict(x_test)

mse_validation = mean_squared_error(y_test, y_val_pred)
r2_scores = r2_score(y_test, y_val_pred)

print("Mean Squared Error on Validation Set:", str(mse_validation) + " ((" + str((mse_validation)**0.5) +"))")
print("R^2 Score on Validation Set:", np.mean(r2_scores))

#12.656, 0.127

        

Mean Squared Error on Validation Set: 160.184446873549 ((12.656399443504815))
R^2 Score on Validation Set: 0.1269480110556248
